In [ ]:
!pip install keras-unet-collection

In [ ]:
import os
import numpy as np
from matplotlib import pyplot as plt
from tensorflow.keras.optimizers import Adam
from keras_unet_collection import models, losses
import tensorflow as tf
from datetime import datetime
import cv2
import random
from PIL import Image

# **Dataset Directory**

In [ ]:
# Define paths to our training, test, and validation Images
train_image_directory = '/kaggle/input/imagedata/DRIVE/training/Original'
test_image_directory = '/kaggle/input/imagedata/DRIVE/test/Original'
validation_image_directory = '/kaggle/input/imagedata/DRIVE/val/Original'

# Define paths to our training, test, and validation Masks
train_masks_directory = '/kaggle/input/imagedata/DRIVE/training/Ground truth'
test_masks_directory = '/kaggle/input/imagedata/DRIVE/test/Ground truth'
validation_masks_directory = '/kaggle/input/imagedata/DRIVE/val/Ground truth'

# **Train dataset**

In [ ]:
train_image_dataset = []
train_mask_dataset = []
SIZE = 224  # Assuming the desired size for the images is 64x64

train_images = os.listdir(train_image_directory)
for i, image_name in enumerate(train_images):
    if image_name.split('.')[1] == 'png':
        image = cv2.imread(os.path.join(train_image_directory, image_name))
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB
        image = Image.fromarray(image)
        image = image.resize((SIZE, SIZE))
        train_image_dataset.append(np.array(image))

train_masks = os.listdir(train_masks_directory)
for i, image_name in enumerate(train_masks):
    if image_name.split('.')[1] == 'png':
        image = cv2.imread(os.path.join(train_masks_directory, image_name), 0)
        image = Image.fromarray(image)
        image = image.resize((SIZE, SIZE))
        train_mask_dataset.append(np.array(image))

# Normalize images
train_image_data = np.array(train_image_dataset) / 255.0
# Rescale masks to 0 to 1
train_mask_data = np.expand_dims(np.array(train_mask_dataset), 3) / 255.0

# **Checking random images from train dataset**

In [ ]:
# Display a random image along with its mask
random_index = random.randint(0, len(train_image_data) - 1)
random_image = train_image_data[random_index]
random_mask = train_mask_data[random_index]

plt.figure(figsize=(6, 4))

# Display the random image
plt.subplot(1, 2, 1)
plt.imshow(random_image)
plt.title('Random Image',fontdict={'family': 'serif', 'size': 12, 'weight': 'bold'})
plt.axis('off')

# Display the corresponding mask
plt.subplot(1, 2, 2)
plt.imshow(random_mask[:, :, 0], cmap='gray')  # Assuming the mask is a single-channel image
plt.title('Corresponding Mask',fontdict={'family': 'serif', 'size': 12, 'weight': 'bold'})
plt.axis('off')

plt.tight_layout()
plt.show()

# **Test Dataset**

In [ ]:
test_image_dataset = []
test_mask_dataset = []
SIZE = 224  # Assuming the desired size for the images is 64x64

test_images = os.listdir(test_image_directory)
for i, image_name in enumerate(test_images):
    if image_name.split('.')[1] == 'png':
        image = cv2.imread(os.path.join(test_image_directory, image_name))
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB
        image = Image.fromarray(image)
        image = image.resize((SIZE, SIZE))
        test_image_dataset.append(np.array(image))

test_masks = os.listdir(test_masks_directory)
for i, image_name in enumerate(test_masks):
    if image_name.split('.')[1] == 'png':
        image = cv2.imread(os.path.join(test_masks_directory, image_name), 0)
        image = Image.fromarray(image)
        image = image.resize((SIZE, SIZE))
        test_mask_dataset.append(np.array(image))

# Normalize images
test_image_data = np.array(test_image_dataset) / 255.0
# Rescale masks to 0 to 1
test_mask_data = np.expand_dims(np.array(test_mask_dataset), 3) / 255.0

# **Checking random images from test dataset**

In [ ]:
# Display a random image along with its mask
random_index = random.randint(0, len(test_image_data) - 1)
random_image = test_image_data[random_index]
random_mask = test_mask_data[random_index]

plt.figure(figsize=(6, 4))

# Display the random image
plt.subplot(1, 2, 1)
plt.imshow(random_image)
plt.title('Random Image',fontdict={'family': 'serif', 'size': 12, 'weight': 'bold'})
plt.axis('off')

# Display the corresponding mask
plt.subplot(1, 2, 2)
plt.imshow(random_mask[:, :, 0], cmap='gray')  # Assuming the mask is a single-channel image
plt.title('Corresponding Mask',fontdict={'family': 'serif', 'size': 12, 'weight': 'bold'})
plt.axis('off')

plt.tight_layout()
plt.show()

# **Validation Dataset**

In [ ]:
validation_image_dataset = []
validation_mask_dataset = []
SIZE = 224  # Assuming the desired size for the images is 64x64

validation_images = os.listdir(test_image_directory)
for i, image_name in enumerate(validation_images):
    if image_name.split('.')[1] == 'png':
        image = cv2.imread(os.path.join(test_image_directory, image_name))
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB
        image = Image.fromarray(image)
        image = image.resize((SIZE, SIZE))
        validation_image_dataset.append(np.array(image))

validation_masks = os.listdir(test_masks_directory)
for i, image_name in enumerate(test_masks):
    if image_name.split('.')[1] == 'png':
        image = cv2.imread(os.path.join(test_masks_directory, image_name), 0)
        image = Image.fromarray(image)
        image = image.resize((SIZE, SIZE))
        validation_mask_dataset.append(np.array(image))

# Normalize images
validation_image_data = np.array(validation_image_dataset) / 255.0
# Rescale masks to 0 to 1
validation_mask_data = np.expand_dims(np.array(validation_mask_dataset), 3) / 255.0

# **Checking random images from validation dataset**

In [ ]:
# Display a random image along with its mask
random_index = random.randint(0, len(validation_image_data) - 1)
random_image = validation_image_data[random_index]
random_mask = validation_mask_data[random_index]

plt.figure(figsize=(6, 4))

# Display the random image
plt.subplot(1, 2, 1)
plt.imshow(random_image)
plt.title('Random Image',fontdict={'family': 'serif', 'size': 12, 'weight': 'bold'})
plt.axis('off')

# Display the corresponding mask
plt.subplot(1, 2, 2)
plt.imshow(random_mask[:, :, 0], cmap='gray')  # Assuming the mask is a single-channel image
plt.title('Corresponding Mask',fontdict={'family': 'serif', 'size': 12, 'weight': 'bold'})
plt.axis('off')

plt.tight_layout()
plt.show()

# **Segmnetation Model: Swin-UNET**


In [ ]:
help(models.swin_unet_2d)

In [ ]:
###############################################################################
IMG_HEIGHT = 224
IMG_WIDTH  = 224
IMG_CHANNELS = 3
num_labels = 1  #Binary
input_shape = (IMG_HEIGHT,IMG_WIDTH,IMG_CHANNELS)
batch_size = 1

model_SwinUNET = models.swin_unet_2d(input_shape, filter_num_begin=64, n_labels=num_labels, depth=4, stack_num_down=2, stack_num_up=2,
                            patch_size=(2, 2), num_heads=[4, 8, 8, 8], window_size=[4, 2, 2, 2], num_mlp=512,
                            output_activation='Sigmoid', shift_window=True, name='swin_unet')

model_SwinUNET.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate = 0.001),
              metrics=['accuracy', losses.dice_coef])

print(model_SwinUNET.summary())

# **Training starts here...**

In [ ]:
from keras.callbacks import ModelCheckpoint

# Specify the file path where the weights will be saved
checkpoint_filepath = '/kaggle/working/your_model_weights.h5'

# Create a ModelCheckpoint callback to save only the weights
model_checkpoint = ModelCheckpoint(
    filepath=checkpoint_filepath,
    monitor='val_dice_coef',  # You can change this to 'val_accuracy' or any other metric
    save_best_only=True,  # Save only the best weights
    save_weights_only=True,  # Save only the model weights, not the entire model
    mode='max',  # 'min' for loss, 'max' for accuracy, 'auto' to infer automatically
    verbose=1
)


In [ ]:
start1 = datetime.now()

SwinUNET_history = model_SwinUNET.fit(train_image_data, train_mask_data,
                    verbose=1,
                    batch_size = batch_size,
                    validation_data=(test_image_data, test_mask_data),
                    shuffle=False,
                    callbacks=[model_checkpoint],                  
                    epochs=100)

stop1 = datetime.now()
#Execution time of the model
execution_time_Unet = stop1-start1
print("UNet execution time is: ", execution_time_Unet)


# **Saved the trained model for the 1st time**

In [ ]:
from tensorflow.keras.models import load_model
# Assuming 'model_TransUNET' is our trained UNet model
model_SwinUNET.save('/kaggle/working/fundus_SwinUNET_model_checkpoint.hdf5')

In [ ]:
model_SwinUNET.save_weights("model.h5")
print("Saved model to disk")

# **Load the trained model for training again**

In [ ]:

# To load the saved model later
# loaded_model = model_SwinUNET.load_weights('/kaggle/working/your_model_weights.h5')
model_old_path = '/kaggle/working/fundus_SwinUNET_model_checkpoint.hdf5'

In [ ]:
# evaluate loaded model on test data
model_SwinUNET.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate = 0.001),
              metrics=['accuracy', losses.dice_coef])

In [ ]:
model_SwinUNET.load_weights(model_old_path)

# **Plotting Accuracy vs Loss Graph**

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
from matplotlib.legend_handler import HandlerLine2D
import numpy as np

# Plot training & validation loss values
plt.figure(figsize=(8, 5))

#Check history plots, one model at a time
history = SwinUNET_history
# Plot Loss
train_loss, = plt.plot(history.history['loss'], label='Train Loss', color='blue')
val_loss, = plt.plot(history.history['val_loss'], label='Validation Loss', color='orange')
train_accuracy, = plt.plot(history.history['accuracy'], label='Train Accuracy',  color='green')
val_accuracy, = plt.plot(history.history['val_accuracy'], label='Validation Accuracy', color='red')


# Add a title with specified font properties
plt.title('Model Performance during Training', fontdict={'family': 'Serif', 'weight': 'bold', 'size': 12},pad=10)
# Set x-axis label with specified font properties
plt.xlabel('No. of Epochs', fontdict={'family': 'Serif', 'weight': 'bold', 'size': 12})

# Set x-axis ticks font properties
plt.xticks(np.linspace(0, len(history.history['loss']), num=6), fontname='Serif', weight='bold')

# Set y-axis ticks font properties
plt.yticks(np.linspace(0.2, 1, num=5), fontname='Serif', weight='bold')

# Set the x-axis and y-axis limits
plt.xlim(0, len(history.history['loss']))
plt.ylim(0, 1)

# Define custom legend lines with desired line properties
legend_lines = [
    Line2D([0], [0], color='blue', lw=3),          # Train Loss
    Line2D([0], [0], color='orange', lw=3),       # Validation Loss
    Line2D([0], [0], color='green', lw=3),        # Train Accuracy
    Line2D([0], [0], color='red', lw=3)           # Validation Accuracy
]

# Place legend outside the graph by adjusting bbox_to_anchor and specifying it to be outside the axes
plt.legend(legend_lines, ['Train Loss', 'Validation Loss', 'Train Accuracy', 'Validation Accuracy'],
           loc='lower center', bbox_to_anchor=(0.5, 1.1), ncol=5,
           prop={'family': 'Serif', 'weight': 'bold', 'size': 8}, frameon=False,
           handler_map={Line2D: HandlerLine2D(numpoints=5)})

# Adjust padding between x-axis label and x-axis ticks
plt.gca().xaxis.labelpad = 10  # Change the value as needed to adjust the space


# Display gridlines for better readability
plt.grid(True)
plt.show()


# **Checking prediction on images from Test Dataset**

In [ ]:
import random
import matplotlib.pyplot as plt

model = model_SwinUNET

test_img_number = 1  # Change the index number here...

test_img = test_image_data[test_img_number]
ground_truth = test_mask_data[test_img_number]
test_img_input = np.expand_dims(test_img, 0)
prediction = (model.predict(test_img_input)[0, :, :, 0] > 0.5).astype(np.uint8)

fig, axs = plt.subplots(1, 3, figsize=(12, 4))

axs[0].imshow(test_img)
axs[0].set_title('Original', fontdict={'family': 'serif', 'size': 12, 'weight': 'bold'})
axs[0].axis('off')

axs[1].imshow(ground_truth[:, :, 0], cmap='gray')
axs[1].set_title('Ground Truth', fontdict={'family': 'serif', 'size': 12, 'weight': 'bold'})
axs[1].axis('off')

prediction_visual = axs[2].imshow(prediction, cmap='gray')
axs[2].set_title('Predicted Mask', fontdict={'family': 'serif', 'size': 12, 'weight': 'bold'})
axs[2].axis('off')
# Save the plotted images
plt.savefig('/kaggle/working/sw_FIVES_prediction_images1_.pdf')  # Modify the file name as needed
plt.show()


# **Segmentation Evaluation Metrics**
* Intersection over Union (IoU)
* Dice Coefficient
* Pixel Accuracy
* Surface Dice Overlap
* Modified Hausdorff Distance

In [ ]:
import tensorflow as tf
from tensorflow.keras import backend as K

def m_iou(y_true, y_pred):
    y_true_float = K.cast(y_true, dtype=tf.float32)
    y_pred_float = K.cast(K.round(y_pred), dtype=tf.float32)

    intersection = K.sum(K.abs(y_true_float * y_pred_float))
    union = K.sum(y_true_float) + K.sum(y_pred_float) - intersection
    iou = intersection / (union + K.epsilon())
    
    return iou

In [ ]:
def dice_coeff(y_true, y_pred):
    y_true_float = K.cast(y_true, dtype=tf.float32)
    y_pred_float = K.cast(K.round(y_pred), dtype=tf.float32)
    
    intersection = K.sum(y_true_float * y_pred_float)
    union = K.sum(y_true_float) + K.sum(y_pred_float)
    dice = (2. * intersection) / (union + K.epsilon())
    return dice

In [ ]:
from sklearn.metrics import precision_recall_fscore_support
from skimage.measure import label, regionprops
from scipy.spatial.distance import directed_hausdorff

# Initialize lists to store evaluation metrics for each image
IoU_values = []
dice_coefficient_values = []
pixel_accuracy_values = []
modified_hausdorff_distance_values = []

n_classes = 2  # Assuming binary segmentation

for img in range(0, test_image_data.shape[0]):
    temp_img = test_image_data[img]
    ground_truth = test_mask_data[img]
    temp_img_input = np.expand_dims(temp_img, 0)
    prediction = (model.predict(temp_img_input)[0, :, :, 0] > 0.5).astype(np.uint8)

    y_true= ground_truth[:, :, 0]
    y_pred=prediction
    y_true_float = K.cast(y_true, dtype=tf.float32)
    y_pred_float = K.cast(K.round(y_pred), dtype=tf.float32)
    # Intersection over Union (IoU)
    intersection = K.sum(y_true_float * y_pred_float)
    
    iou = m_iou(ground_truth[:, :, 0], prediction)
    IoU_values.append(iou)

    # # Dice Coefficient
    dice = dice_coeff(ground_truth[:, :, 0], prediction)
    dice_coefficient_values.append(dice)
    # Dice Coefficient
    #dice = (2 * np.sum(intersection)) / (np.sum(ground_truth[:, :, 0]) + np.sum(prediction))
    dice_coefficient_values.append(dice)
    
    # Pixel Accuracy
    pixel_accuracy = np.sum(intersection) / np.sum(ground_truth[:, :, 0])
    pixel_accuracy_values.append(pixel_accuracy)
    
    

    # Modified Hausdorff Distance
    hausdorff_distance = directed_hausdorff(ground_truth[:, :, 0], prediction)[0]
    modified_hausdorff_distance_values.append(hausdorff_distance)

# Calculate mean values for all metrics
mean_IoU = np.mean(IoU_values)
mean_dice_coefficient = np.mean(dice_coefficient_values)
mean_pixel_accuracy = np.mean(pixel_accuracy_values)
mean_modified_hausdorff_distance = np.mean(modified_hausdorff_distance_values)


In [ ]:
surface_dice_overlap_values = []

n_classes = 2  # Assuming binary segmentation

for img in range(0, test_image_data.shape[0]):
    temp_img = test_image_data[img]
    ground_truth = test_mask_data[img]
    temp_img_input = np.expand_dims(temp_img, 0)
    prediction = (model.predict(temp_img_input)[0, :, :, 0] > 0.5).astype(np.uint8)

    # Surface Dice Overlap (Assuming binary images)
    labeled_true = label(ground_truth[:, :, 0])
    labeled_pred = label(prediction)
    props_true = regionprops(labeled_true)
    props_pred = regionprops(labeled_pred)

    # Calculate Surface Dice Overlap for each pair of regions
    dice_overlap_sum = 0
    pairs_count = 0

    for p in props_true:
        for q in props_pred:
            intersection = np.logical_and(labeled_true == p.label, labeled_pred == q.label)
            if np.sum(intersection) > 0:
                dice_overlap = 2 * np.sum(intersection) / (np.sum(labeled_true == p.label) + np.sum(labeled_pred == q.label))
                dice_overlap_sum += dice_overlap
                pairs_count += 1

    surface_dice_overlap = dice_overlap_sum / pairs_count if pairs_count > 0 else 0
    surface_dice_overlap_values.append(surface_dice_overlap)

# Calculate mean value for surface dice overlap
mean_surface_dice_overlap = np.mean(surface_dice_overlap_values)



# **Print mean values of Evaluation metrics**

In [ ]:
# Print mean values
print("Mean Intersection over Union (IoU):", mean_IoU)
print("Mean Dice Coefficient:", mean_dice_coefficient)
print("Mean Pixel Accuracy:", mean_pixel_accuracy)
print("Mean Modified Hausdorff Distance:", mean_modified_hausdorff_distance)
print("Mean Surface Dice Overlap:", mean_surface_dice_overlap)